In [1]:
import os, numpy as np, regex, pandas, pickle
from collections import defaultdict
jox_root = '/your_jox_root/'
dir_pattern = jox_root+'JOx_FMX_xx_yyyy/txt/'
langs = ('HU','EN')

In [78]:
def read_document(fpath,tokens,file_length_list,cell_count_list):
    f = open(fpath,encoding = 'utf8')
    txt = f.read()
    f.close()
    tokens.append(np.array([w for w in regex.split('(\W)',txt) if len(w) and not w.isspace()]))
    file_length_list.append(len(txt))
    cell_count_list.append(len(regex.findall('\|',txt)))

first_year = 2004
last_year = 2006

for yr in range(first_year,last_year):
    dirname = dir_pattern.replace('xx',langs[0]).replace('yyyy',str(yr))
    os.chdir(dirname)
    fl = os.listdir()
    keep_files = list()
    file_length_list = (list(),list())
    token_count_list = (list(),list())
    cell_count_list = (list(),list())
    numeric_count_list = (list(),list())
    alpha_count_list = (list(),list())
    alpha_length_list = (list(),list())
    df = list()
    for fn in fl[:100]:
        tokens = list()
        fpath = dirname + fn
        read_document(fpath,tokens,file_length_list[0],cell_count_list[0])
        if len(tokens[0]) < 10:
            file_length_list[0].pop()
            cell_count_list[0].pop()
            continue
        keep_files.append(fn)
        read_document(fpath.replace(langs[0],langs[1]),tokens,file_length_list[1],cell_count_list[1])
        for i in range(len(langs)):
            token_count_list[i].append(len(tokens[i]))
            numeric_count_list[i].append(sum(np.char.isnumeric(tokens[i])))
            alpha_count_list[i].append(sum(np.char.isalpha(tokens[i])))
            alpha_length_list[i].append(sum(np.char.str_len(np.array(tokens[i])[np.char.isalpha(tokens[i])])))

    for i in range(len(langs)):
        if i > 0:
            keep_files = [f.replace(langs[i-1],langs[i]) for f in keep_files]
        df.append(pandas.DataFrame({'file_name': keep_files, 'num_chars': file_length_list[i],
                                    'num_tokens': token_count_list[i], 'num_table_cells': cell_count_list[i],
                                    'num_numeric_tokens': numeric_count_list[i],
                                    'num_alphabetic_tokens': alpha_count_list[i],
                                    'length_alphabetic_tokens': alpha_length_list[i]}))
    
    f = open(jox_root+'%s.pickle' % str(yr),'wb')
    pickle.dump(df,f)
    f.close()

In [268]:
(first_year,last_year) = (2004,2020)
dfs = {}
for yr in range(first_year,last_year):
    f = open(jox_root+'%s.pickle' % str(yr),'rb')
    dfs[yr] = pickle.load(f)
    f.close()

In [269]:
dfs[2004][0]['num_tokens'].min()

23

In [45]:
filecounts = pandas.DataFrame(np.zeros([16,4]),columns=['year','all_files','filtered_files','diff'],dtype=('int'))
for yr in range(first_year,last_year):
    dirname = dir_pattern.replace('xx',langs[0]).replace('yyyy',str(yr))
    filecounts['year'][yr-first_year] = yr
    filecounts['all_files'][yr-first_year] = len(os.listdir(dirname))
    filecounts['filtered_files'][yr-first_year] = len(dfs[yr][0])

filecounts

,year,all_files,filtered_files,diff
0,2004,5675,5675,0
1,2005,11964,11964,0
2,2006,14639,14639,0
3,2007,11409,11409,0
4,2008,13337,13337,0
5,2009,12049,12049,0
6,2010,12476,12476,0
7,2011,12499,12499,0
8,2012,12029,12029,0
9,2013,12879,12879,0


In [46]:
filecounts['diff'] = filecounts['all_files'] - filecounts['filtered_files']

In [50]:
minfiles = pandas.DataFrame(np.zeros([16,3]),columns=['year','min_HU','min_EN'],dtype=('int'))

In [51]:
for yr in range(first_year,last_year):
    dirname = dir_pattern.replace('xx',langs[0]).replace('yyyy',str(yr))
    minfiles['year'][yr-first_year] = yr
    minfiles['min_HU'][yr-first_year] = dfs[yr][0]['num_tokens'].min()
    minfiles['min_EN'][yr-first_year] = dfs[yr][1]['num_tokens'].min()
minfiles

,year,min_HU,min_EN
0,2004,23,26
1,2005,26,24
2,2006,24,25
3,2007,23,25
4,2008,25,22
5,2009,23,24
6,2010,24,21
7,2011,26,25
8,2012,26,26
9,2013,22,24


In [90]:
dfs[2010][0].columns

Index(['file_name', 'num_chars', 'num_tokens', 'num_table_cells',
       'num_numeric_tokens', 'num_alphabetic_tokens',
       'length_alphabetic_tokens'],
      dtype='object')

In [194]:
ratios = pandas.DataFrame()
for yr in range(first_year,last_year):
    hu_data = dfs[yr][0]
    en_data = dfs[yr][1]
    ratios = ratios.append(
        pandas.DataFrame(
        {'year': [yr]*len(hu_data),
            'file_name': hu_data['file_name'],
           'num_alpha_ratio': hu_data['num_numeric_tokens'] / hu_data['num_alphabetic_tokens'],
         'hu_en_num_ratio': hu_data['num_numeric_tokens'] / en_data['num_numeric_tokens'],
         'hu_en_length_ratio': hu_data['num_chars'] / en_data['num_chars'],
         'hu_en_char_ratio': (hu_data['num_alphabetic_tokens'] + hu_data['length_alphabetic_tokens']) / 
             (en_data['num_alphabetic_tokens'] + en_data['length_alphabetic_tokens']),
        })
    )

In [238]:
ratios.to_csv(jox_root+'ratios.csv')

In [191]:
ratios['hu_en_length_ratio'][ratios['num_alpha_ratio'] < 0.3].describe(percentiles=[0.01,0.1,0.25,0.5,0.75,0.9,0.99])

count    158206.000000
mean          1.054561
std           0.671184
min           0.009905
1%            0.894446
10%           0.972181
25%           1.004547
50%           1.045210
75%           1.089961
90%           1.131380
99%           1.251316
max         154.484171
Name: hu_en_length_ratio, dtype: float64

In [190]:
ratios['hu_en_char_ratio'][(ratios['num_alpha_ratio'] < 0.3) & (ratios['hu_en_length_ratio'] > 0.89) & (ratios['hu_en_length_ratio'] < 1.25)].describe()

count    155216.000000
mean          1.046779
std           0.067543
min           0.808841
25%           1.000430
50%           1.043381
75%           1.090553
max           1.415254
Name: hu_en_char_ratio, dtype: float64

In [199]:
ratios[(ratios['year'] == 2004) & (ratios['num_alpha_ratio'] < 0.3) & (ratios['hu_en_length_ratio'] > 0.89) & (ratios['hu_en_length_ratio'] < 1.25)]

,year,file_name,num_alpha_ratio,hu_en_num_ratio,hu_en_length_ratio,hu_en_char_ratio
0,2004,C_2004229HU.01001602.xml.txt,0.080645,1.000000,1.019567,1.016911
5,2004,CE2004321HU.01000602.xml.txt,0.023098,1.000000,1.063117,1.060715
8,2004,C_2004161HU.01000901.xml.txt,0.162791,1.000000,0.998668,0.998494
10,2004,C_2004314HU.01000101.xml.txt,0.158730,1.000000,1.049312,1.042724
11,2004,L_2004173HU.01000701.xml.txt,0.152542,1.000000,1.030722,1.046135
12,2004,L_2004172HU.01000101.xml.txt,0.214724,1.000000,1.010309,1.005426
13,2004,L_2004265HU.01000501.xml.txt,0.279605,1.000000,1.042194,1.039527
15,2004,C_2004300HU.01005401.xml.txt,0.048860,1.071429,0.962380,0.954473
16,2004,L_2004256HU.01002101.xml.txt,0.243590,1.085714,0.997416,0.968912
19,2004,C_2004218HU.01000201.xml.txt,0.193548,1.200000,1.020619,1.010062


In [206]:
ratios = pandas.DataFrame.read_csv(jox_root+'ratios.csv')

In [207]:
len(fajlok)

4166

In [244]:
import time
(first_year,last_year) = (2004,2006)
df = pandas.DataFrame()
for yr in range(first_year,last_year):
    print(yr,'start')
    dirname = dir_pattern.replace('xx','HU').replace('yyyy',str(yr))
    fajlok = ratios['file_name'][(ratios['year'] == yr) & (ratios['num_alpha_ratio'] < 0.3) & (ratios['hu_en_length_ratio'] > 0.89) & (ratios['hu_en_length_ratio'] < 1.25)].to_numpy()
    start_time = time.time()
    similarity = np.zeros(len(fajlok))
    for i in range(len(fajlok)):
        alpha_types = list()
        fpath = dirname + fajlok[i]
        f = open(fpath,encoding = 'utf8')
        txt = f.read()
        f.close()
        alpha_types.append(set([w for w in regex.split('(\W)',txt) if len(w) and w.isalpha()]))
        f = open(fpath.replace('HU','EN'),encoding = 'utf8')
        txt = f.read()
        f.close()
        alpha_types.append(set([w for w in regex.split('(\W)',txt) if len(w) and w.isalpha()]))
        similarity[i] = len(alpha_types[0].intersection(alpha_types[1])) / min(len(alpha_types[0]),len(alpha_types[1]))
    df = df.append(pandas.DataFrame({'year': [yr] * len(fajlok), 'file_name': fajlok, 'similarity': similarity}))
    print(yr,'done')
    print(time.time()-start_time)

2004 start
2004 done
8.406369924545288
2005 start
2005 done
22.29979681968689


In [254]:
df2 = pandas.merge(ratios,df[['file_name','similarity']],on='file_name')

In [ ]:
df2.to_csv(jox_root+'ratios_with_en_hu_similarity.csv')

In [ ]:
df2['similarity'].describe(percentiles=[0.01,0.1,0.25,0.5,0.75,0.9,0.99])

In [432]:
df2['file_name'][df2['similarity'] < 0.5].to_csv('/home/pgergo/prog/msc/JOx/files_filtered_by_similarity.csv')

C:\Users\Gergo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [259]:
filtered_files = pandas.read_csv(jox_root+'files_filtered_by_similarity.csv',header=None)

In [263]:
filtered_files.columns = [0,'file_name']

In [265]:
filtered_files = filtered_files['file_name']

In [272]:
len(pandas.merge(dfs[2004][0],filtered_files))

3825

In [357]:
sorted_filtered = pandas.merge(dfs[2004][0],filtered_files).sort_values(by=['num_chars'])
sorted_filtered_np = sorted_filtered[['file_name','num_chars']].to_numpy()

In [343]:
comparisons = 0
for i in range(1999):
    j = i + 1
    while j < len(sorted_filtered_np) and sorted_filtered_np[j][1] < sorted_filtered_np[i][1] * 1.2:
        comparisons += 1
        j += 1
comparisons

670200

In [344]:
dirname = dir_pattern.replace('xx',langs[0]).replace('yyyy',str(2004))
alpha_types = list()
for i in range(2000):
    fpath = dirname + sorted_filtered_np[i][0]
    f = open(fpath,encoding = 'utf8')
    txt = f.read()
    f.close()
    alpha_types.append(set([w for w in regex.split('(\W)',txt) if len(w) and w.isalpha()]))

In [345]:
len(alpha_types)

2000

In [358]:
comparisons = 0
is_duplicate_of = {}
for i in range(1999):
    j = i + 1
    while j < 2000 and sorted_filtered_np[j][1] < sorted_filtered_np[i][1] * 1.2:
        comparisons += 1
        overlap = len(alpha_types[i].intersection(alpha_types[j])) / min(len(alpha_types[i]),len(alpha_types[j]))
        if overlap > 0.85:
            is_duplicate_of[sorted_filtered_np[i][0]] = (sorted_filtered_np[j][0],overlap)
            break
        j += 1
comparisons

355765

In [543]:
len(is_duplicate_of)

765

In [ ]:
import time
(first_year,last_year) = (2004,2006)
is_duplicate_of = {}
for yr in range(first_year,last_year):
    print(yr,'start')
    dirname = dir_pattern.replace('xx','HU').replace('yyyy',str(yr))
    sorted_filtered = pandas.merge(dfs[yr][0],filtered_files).sort_values(by=['num_chars'])
    sorted_filtered_np = sorted_filtered[['file_name','num_chars']].to_numpy()
    start_time = time.time()
    
    alpha_types = list()
    for item in sorted_filtered_np:
        fpath = dirname + item[0]
        f = open(fpath,encoding = 'utf8')
        txt = f.read()
        f.close()
        alpha_types.append(set([w for w in regex.split('(\W)',txt) if len(w) and w.isalpha()]))
    
    comparisons = 0
    for i in range(len(sorted_filtered_np) - 1):
        j = i + 1
        while j < len(sorted_filtered_np) and sorted_filtered_np[j][1] < sorted_filtered_np[i][1] * 1.2:
            comparisons += 1
            overlap = len(alpha_types[i].intersection(alpha_types[j])) / min(len(alpha_types[i]),len(alpha_types[j]))
            if overlap > 0.85:
                is_duplicate_of[sorted_filtered_np[i][0]] = (sorted_filtered_np[j][0],overlap)
                break
            j += 1

    print(comparisons,'comparisons')
    print(yr,'done')
    print(time.time()-start_time)

In [434]:
filecounts['all_files'].sum()

186434

In [435]:
len(dfs)

16

In [486]:
total_en = 0
total_hu = 0
total_length = 0
for y in dfs:
    total_length += len(dfs[y][0])
    total_hu += dfs[y][0]['num_tokens'].sum()
    total_en += dfs[y][1]['num_tokens'].sum()

In [562]:
total_length

110651

In [563]:
total_en

207315922

In [564]:
total_hu

190981860

In [494]:
total_en = 0
total_hu = 0
total_length = 0
for y in dfs:
    cc = pandas.concat([pandas.merge(dfs[y][0],ratios,on='file_name'),dfs[y][1]],1)
    cc_filtered = cc[(cc['num_alpha_ratio'] < 0.3) & (cc['hu_en_length_ratio'] > 0.89) & (cc['hu_en_length_ratio'] < 1.25)]
    total_length += len(cc_filtered)
    (hu, en) = cc_filtered['num_tokens'].to_numpy().sum(0)
    total_hu += hu
    total_en += en

In [539]:
total_en = 0
total_hu = 0
total_length = 0
for y in dfs:
    filtered = pandas.merge(
        dfs[y][0].join(dfs[y][1],lsuffix='_hu',rsuffix='_en'),
        filtered_files,
        left_on='file_name_hu', right_on='file_name')
    total_length += len(filtered)
    (hu, en) = filtered[['num_tokens_hu','num_tokens_en']].sum(0)
    total_hu += hu
    total_en += en

In [544]:
f = open(jox_root+'duplicate_files.pickle','rb')
duplicates = pickle.load(f)
f.close()

In [545]:
dupe_names = set(duplicates.keys())
all_names = set(filtered_files['file_name'])
keep_names = pandas.DataFrame({'file_name': list(all_names - dupe_names)})

35584

In [561]:
total_en = 0
total_hu = 0
total_length = 0
for y in dfs:
    final = pandas.merge(
        dfs[y][0].join(dfs[y][1],lsuffix='_hu',rsuffix='_en'),
        keep_names,
        left_on='file_name_hu', right_on='file_name')
    total_length += len(final)
    (hu, en) = final[['num_tokens_hu','num_tokens_en']].sum(0)
    total_hu += hu
    total_en += en

In [560]:
len(final)

3793

In [577]:
keep_names.shape

(110651, 1)

In [552]:
len(all_names)

146235

In [575]:
keep_names.to_csv(jox_root+'keep_files.csv')

In [558]:
len(keep_names)

110651

In [576]:
ratios.shape

(186434, 6)

In [578]:
ratios_final = pandas.merge(ratios,keep_names,on='file_name')

In [579]:
ratios_final.shape

(110651, 6)

In [580]:
ratios_final['hu_en_char_ratio'].mean()

1.0524100328857948

In [585]:
ratios_final['hu_en_char_ratio'].std()

0.06806975428391841

In [586]:
ratios.columns

Index(['year', 'file_name', 'num_alpha_ratio', 'hu_en_num_ratio',
       'hu_en_length_ratio', 'hu_en_char_ratio'],
      dtype='object')

In [589]:
dfs[2004][0].columns

Index(['file_name', 'num_chars', 'num_tokens', 'num_table_cells',
       'num_numeric_tokens', 'num_alphabetic_tokens',
       'length_alphabetic_tokens'],
      dtype='object')

In [713]:
ossz = pandas.DataFrame()
for yr in range(2004,2020):
    ossz = ossz.append(dfs[yr][0].join(dfs[yr][1],lsuffix='_hu',rsuffix='_en'))[['file_name_hu','num_chars_hu','num_numeric_tokens_hu','num_alphabetic_tokens_hu',
       'length_alphabetic_tokens_hu','num_alphabetic_tokens_en','length_alphabetic_tokens_en','num_chars_en','num_numeric_tokens_en']]

C:\Users\Gergo\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [638]:
dfs[2008][1].shape

(13337, 7)

In [714]:
ossz.shape

(186434, 9)

In [611]:
dfs[2004][0].join(dfs[2004][1],lsuffix='_hu',rsuffix='_en').shape

(5675, 14)

In [612]:
ossz = pandas.DataFrame()

In [613]:
ossz.append(dfs[2004][0].join(dfs[2004][1],lsuffix='_hu',rsuffix='_en')).shape

(5675, 14)

In [646]:
ossz.shape

(186434, 5)

In [623]:
ossz.columns

Index(['file_name_hu', 'num_chars_hu', 'num_tokens_hu', 'num_table_cells_hu',
       'num_numeric_tokens_hu', 'num_alphabetic_tokens_hu',
       'length_alphabetic_tokens_hu', 'file_name_en', 'num_chars_en',
       'num_tokens_en', 'num_table_cells_en', 'num_numeric_tokens_en',
       'num_alphabetic_tokens_en', 'length_alphabetic_tokens_en'],
      dtype='object')

In [716]:
final_properties = pandas.merge(ossz,ratios_final,left_on='file_name_hu',right_on='file_name')

In [688]:
final_properties[(1200 < final_properties['num_chars_hu']) & (final_properties['num_chars_hu'] < 1300)]['hu_en_char_ratio'].std()

0.07089374526224779

In [689]:
len(df2)

155216

In [690]:
df2.shape

(155216, 8)

In [694]:
pandas.merge(df2,keep_names,on='file_name')['similarity']

0         0.328571
1         0.341463
2         0.049296
3         0.061111
4         0.204545
5         0.102564
6         0.120690
7         0.050360
8         0.051282
9         0.036866
10        0.222222
11        0.077176
12        0.031818
13        0.066176
14        0.208333
15        0.028926
16        0.036145
17        0.059259
18        0.056911
19        0.060995
20        0.033149
21        0.111717
22        0.378378
23        0.145985
24        0.134228
25        0.080537
26        0.027972
27        0.122905
28        0.061644
29        0.180645
            ...   
110621    0.022005
110622    0.233831
110623    0.263415
110624    0.059783
110625    0.246684
110626    0.177215
110627    0.363636
110628    0.102564
110629    0.099237
110630    0.058252
110631    0.085174
110632    0.077381
110633    0.026616
110634    0.051282
110635    0.160494
110636    0.098039
110637    0.024691
110638    0.097087
110639    0.095000
110640    0.042969
110641    0.076923
110642    0.

In [696]:
import scipy.stats
fit_alpha, fit_loc, fit_beta=scipy.stats.gamma.fit(pandas.merge(df2,keep_names,on='file_name')['similarity'])

In [697]:
fit_alpha, fit_loc, fit_beta

(1.674982055374307, -0.0008541176512601694, 0.06827457109777044)

In [707]:
scipy.stats.gamma.cdf(0.07, fit_alpha, loc=fit_loc, scale=fit_beta)

0.38009490203353136

In [717]:
final_properties.columns

Index(['file_name_hu', 'num_chars_hu', 'num_numeric_tokens_hu',
       'num_alphabetic_tokens_hu', 'length_alphabetic_tokens_hu',
       'num_alphabetic_tokens_en', 'length_alphabetic_tokens_en',
       'num_chars_en', 'num_numeric_tokens_en', 'year', 'file_name',
       'num_alpha_ratio', 'hu_en_num_ratio', 'hu_en_length_ratio',
       'hu_en_char_ratio'],
      dtype='object')

In [712]:
ossz.columns

Index(['file_name_hu', 'num_chars_hu', 'num_numeric_tokens_hu', 'num_chars_en',
       'num_numeric_tokens_en'],
      dtype='object')

In [718]:
final_properties.shape

(110651, 15)

In [740]:
non_alpha_ratio_hu_en = (final_properties['num_chars_hu']-final_properties['length_alphabetic_tokens_hu']-final_properties['num_alphabetic_tokens_hu']+1) / (final_properties['num_chars_en']-final_properties['length_alphabetic_tokens_en']-final_properties['num_alphabetic_tokens_en']+1)

In [741]:
non_alpha_ratio_hu_en[non_alpha_ratio_hu_en == np.inf]

Series([], dtype: float64)

In [742]:
pandas.DataFrame(non_alpha_ratio_hu_en).describe()

,0
count,110651.000000
mean,1.092143
std,0.110825
min,0.200000
25%,1.034859
50%,1.079618
75%,1.133333
max,15.000000


In [746]:
final_properties['hu_en_char_ratio'].describe()

count    110651.000000
mean          1.052410
std           0.068070
min           0.808841
25%           1.006548
50%           1.049925
75%           1.094538
max           1.415254
Name: hu_en_char_ratio, dtype: float64

In [747]:
final_properties['hu_en_length_ratio'].describe()

count    110651.000000
mean          1.054266
std           0.062000
min           0.890052
25%           1.012346
50%           1.051975
75%           1.093331
max           1.249924
Name: hu_en_length_ratio, dtype: float64

In [803]:
sorted_lengths_ratios = final_properties[['hu_en_length_ratio','num_chars_hu']].sort_values(by='num_chars_hu')
sorted_lengths_ratios = sorted_lengths_ratios.join(pandas.DataFrame({'log_of_length_ratio': np.log(sorted_lengths_ratios['hu_en_length_ratio'])}))

In [804]:
sorted_lengths_ratios[:10]

,hu_en_length_ratio,num_chars_hu,log_of_length_ratio
14297,0.938931,123,-0.063013
3985,1.007812,129,0.007782
108911,1.200000,132,0.182322
69127,1.000000,132,0.000000
24506,0.933333,140,-0.068993
18725,1.217391,140,0.196710
55656,1.076336,141,0.073563
31109,1.036765,141,0.036105
107195,1.234783,142,0.210895
21931,0.959459,142,-0.041385


In [813]:
num_slices = 500
length_ratio_statistics = pandas.DataFrame({'median_length': np.zeros(num_slices),'mean_length_ratio': np.zeros(num_slices), 'std_deviation_length_ratio': np.zeros(num_slices), 'mean_log_length_ratio': np.zeros(num_slices), 'std_dev_log_length_ratio': np.zeros(num_slices)})

In [814]:
sorted_lengths_ratios.iloc[:10,:].iloc[:,1]

14297     123
3985      129
108911    132
69127     132
24506     140
18725     140
55656     141
31109     141
107195    142
21931     142
Name: num_chars_hu, dtype: int64

In [815]:
length_ratio_statistics['median_length'][0]

0.0

In [816]:
slice_length = int(len(sorted_lengths_ratios)/num_slices)
for i in range(num_slices):
    data_slice = sorted_lengths_ratios.iloc[(i*slice_length):(i+1)*slice_length,:]
    length_ratio_statistics['median_length'][i] = data_slice.iloc[:,1].median()
    length_ratio_statistics['mean_length_ratio'][i] = data_slice.iloc[:,0].mean()
    length_ratio_statistics['std_deviation_length_ratio'][i] = data_slice.iloc[:,0].std()
    length_ratio_statistics['mean_log_length_ratio'][i] = data_slice.iloc[:,2].mean()
    length_ratio_statistics['std_dev_log_length_ratio'][i] = data_slice.iloc[:,2].std()
        

In [819]:
length_ratio_statistics

,median_length,mean_length_ratio,std_deviation_length_ratio,mean_log_length_ratio,std_dev_log_length_ratio
0,168.0,1.077397,0.080069,0.071752,0.075303
1,190.0,1.109347,0.079118,0.101153,0.073192
2,206.0,1.104525,0.073773,0.097130,0.068280
3,220.0,1.101617,0.083504,0.093824,0.077722
4,234.0,1.083372,0.093438,0.076283,0.087875
5,252.0,1.069663,0.093671,0.063488,0.088259
6,271.0,1.064403,0.088272,0.058986,0.083039
7,288.0,1.067958,0.089882,0.062164,0.085252
8,305.0,1.037064,0.099873,0.031792,0.096115
9,323.0,1.033923,0.103842,0.028354,0.100235


In [818]:
length_ratio_statistics.to_csv(jox_root+'ratios_by_length.csv')

In [825]:
x = length_ratio_statistics['median_length'].to_numpy()

In [826]:
y1 = length_ratio_statistics['mean_log_length_ratio'].to_numpy()
y2 = length_ratio_statistics['std_dev_log_length_ratio'].to_numpy()

In [828]:
y = length_ratio_statistics[['mean_log_length_ratio','std_dev_log_length_ratio']].to_numpy()

In [829]:
y.shape

(500, 2)

In [838]:
linear_model = np.polyfit(x[:70],y[:70],1)

In [847]:
linear_model[:,0]

array([-1.66230513e-05,  5.62334540e-02])

In [844]:
-1.66230513e-05 * 600 + 5.62334540e-02

0.046259623220000004

In [845]:
-3.36208230e-05 * 600 + 9.07289957e-02

0.0705565019